In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
embed = hub.load(module_url)

In [3]:
# embed = tf.saved_model.load('embed')

In [4]:
embed

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1c5a5432940>

In [5]:
embed(['you should know that sam is great'])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 5.06537668e-02,  6.89419685e-03,  3.10490131e-02,
        -2.34426893e-02, -1.76232699e-02, -3.86518948e-02,
        -1.59729458e-02, -2.87181232e-03,  2.44737212e-02,
         6.50344416e-03,  2.86781415e-02,  5.03714308e-02,
        -8.89344662e-02,  7.95358494e-02, -5.39800264e-02,
        -2.99837962e-02,  1.23190265e-02,  3.16822827e-02,
        -1.13119120e-02, -9.22781229e-02, -7.93654844e-03,
         6.63103238e-02, -1.30027970e-02,  3.67623977e-02,
        -7.49693140e-02,  5.48161902e-02,  3.63411009e-02,
         2.41148751e-02,  4.06515934e-02, -2.10768916e-02,
        -4.13329937e-02, -5.61750028e-03,  3.05836219e-02,
        -4.46937699e-03, -4.67080772e-02,  7.07993954e-02,
         2.32782234e-02, -4.18055020e-02, -6.91551790e-02,
         7.09726363e-02, -3.77384131e-03, -3.47874351e-02,
         1.78640671e-02,  7.12387264e-02, -4.53207493e-02,
         5.43508632e-03, -1.89015071e-03,  6.34287968e-02,
      

In [24]:
import nltk
from nltk.stem import PorterStemmer
def preprocess(text):
    lst_stop_words = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other",
                      "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
    punctuations = '''!()-[\’\“\”]{—};:'"\,<>./?@#$%^&*_~'''
    ps = PorterStemmer()
    word_lst = text.lower().split()
    new_word_lst = []
    for word in word_lst:
        mod_word = ''.join([char for char in word if char not in punctuations])
        mod_word = ps.stem(mod_word)
        new_word_lst.append(mod_word)
        if mod_word in lst_stop_words:
            new_word_lst.remove(mod_word)
    mod_text = ' '.join(new_word_lst)

    return mod_text


In [25]:
import numpy as np
from scipy.sparse import csr_matrix, hstack, vstack
global feat_vec, feat_svec
feat_vec = np.empty((1025))
feat_vec[:] = np.nan
feat_svec = csr_matrix(feat_vec)
feat_svec, feat_svec.toarray()

(<1x1025 sparse matrix of type '<class 'numpy.float64'>'
 	with 1025 stored elements in Compressed Sparse Row format>,
 array([[nan, nan, nan, ..., nan, nan, nan]]))

In [26]:
def get_feature_vector(user_data, admin_data):
    user_emb = embed([user_data])
    admin_emb = embed([admin_data])
    sim = np.inner(user_emb, admin_emb)
    global feat_svec
    feat_svec = vstack((feat_svec, hstack((admin_emb[0], sim[0], user_emb[0]))))
    feat_svec = feat_svec.tocsr()
    feat_svec = feat_svec[1:]
    feat_vec = feat_svec.toarray()
    print(feat_vec.shape)
    print('Vector:', feat_vec)
    print('similarity:', sim[0])
    return feat_vec, sim[0][0]

In [27]:
from tensorflow import keras
def predict(feature_vec):
    model_1_keras = keras.models.load_model('model_1.keras')
    meaning = {0: 'Agree', 1: 'Disagree', 2: 'Discuss', 3: 'Unrelated'}
    label = np.argmax(model_1_keras.predict(features), axis = -1)
    print(label[0])
    result = meaning[label[0]]
    print('Result:', result)
    return result

In [49]:
from GoogleNews import GoogleNews
from newspaper import Article
def get_admin_data(user_data):
    googlenews = GoogleNews(lang='en')
    googlenews.search(user_data)
    links = googlenews.get__links()
    print(len(links))
    if len(links) == 0:
        return '', '', ''
    elif len(links) == 1:
        link_used = links[0]
    else:
        link_used = links[1]
    
    article = Article(link_used)
    print(link_used)
    article.download()
    article.parse()
    article.nlp()
#     admin_data = article.text
    admin_data = article.summary
    print(admin_data)
    
    img_link = article.top_image
    
    return admin_data, link_used, img_link

In [29]:
# print(get_admin_data('covid 19'))

In [50]:
import os
path = 'C:/Users/LEGION/Documents/fake_news_detection_app/'
# To check if file is empty or not, True if empty, False if not empty
os.stat(path + 'Input.json').st_size == 0

True

In [ ]:
import os
import json
import time
path = 'C:/Users/LEGION/Documents/fake_news_detection_app/'
global features
while(True):
    no_inp = os.stat(path + 'Input.json').st_size == 0
    if no_inp == True:
        pass
    else:
        # Get user input from input.json file
        i = open(path + 'Input.json', 'r+')
        inp_data = json.load(i)
        print(inp_data)
        i.truncate(0)
        i.close()
        
        # Preprocessing user data
        inp_data['content'] = preprocess(inp_data['content'])
        
        # Getting our own data
        admin_data, link, img_link = get_admin_data(inp_data['heading'])
        if(link == ''):
            r = {'result': 'Possibly_Fake', 'similarity': 0, 'admin_link': link, 'admin_data': admin_data, 'image_url': img_link}
            json_data = json.dumps(r)
            f = open(path + 'Output.json', 'w')
            f.write(json_data)
            f.close()
            continue
        
        # Preprocessing our data
        admin_data = preprocess(admin_data)
        
        # Getting Feature Vector
        features, sim = get_feature_vector(inp_data['content'], admin_data)
        
        # Prediction on Feature Vector
        result = predict(features)
        r = {'result': result, 'similarity': sim * 100, 'admin_link': link, 'admin_data': admin_data, 'image_url': img_link}
        
        # Writing result in Output.json file
        json_data = json.dumps(r)
        f = open(path + 'Output.json', 'w')
        f.write(json_data)
        f.close()
        # time.sleep(1)

{'heading': 'Days after Gangster Vikas Dubey Killed in Police Encounter, His Father Dies after Heart Attack', 'content': "he father of gangster Vikas Dubey, who was shot dead in an encounter with the Uttar Pradesh Police last week, died after a heart attack on Monday.  Ram Kumar Dubey died in his daughter's home in Shivli in Kanpur rural. He was reportedly in deep shock after his son's death and had stopped having water and food.  On July 3, eight police personnel, including a DSP, were gunned down in Bikru village by the henchmen of Vikas Dubey. Seven others, including a civilian, were injured in the attack from a rooftop after the police team entered the village past July 2 midnight to arrest the gangster.  Vikas Dubey was killed in an encounter on July 10 after police claimed that he tried to escape from the spot in Bhauti area where the vehicle carrying him from Ujjain to Kanpur met with an accident."}
10
https://www.news18.com/news/india/days-after-gangster-vikas-dubey-killed-in-p

In [ ]:
import os
import json
import time
path = 'C:/Users/LEGION/Documents/fake_news_detection_app/'
global features
while(True):
    no_inp = os.stat(path + 'Input.json').st_size == 0
    if no_inp == True:
        pass
    else:
        # Get user input from input.json file
        i = open(path + 'Input.json', 'r+')
        inp_data = json.load(i)
        print(inp_data)
        i.truncate(0)
        i.close()
        
        # Preprocessing user data
        inp_data['content'] = preprocess(inp_data['content'])
        
        # Getting our own data
        admin_data, link, img_link = get_admin_data(inp_data['heading'])
        if(link == ''):
            r = {'result': 'Possibly_Fake', 'similarity': 0, 'admin_link': link, 'admin_data': admin_data, 'image_url': img_link}
            json_data = json.dumps(r)
            f = open(path + 'Output.json', 'w')
            f.write(json_data)
            f.close()
            continue
        
        # Preprocessing our data
        admin_data = preprocess(admin_data)
        
        # Getting Feature Vector
        features, sim = get_feature_vector(inp_data['content'], admin_data)
        
        # Prediction on Feature Vector
        result = predict(features)
        r = {'result': result, 'similarity': sim * 100, 'admin_link': link, 'admin_data': admin_data, 'image_url': img_link}
        
        # Writing result in Output.json file
        json_data = json.dumps(r)
        f = open(path + 'Output.json', 'w')
        f.write(json_data)
        f.close()
        # time.sleep(1)

In [50]:
os.stat(path + 'Input.json').st_size == 0

True

In [46]:
features

array([[ 0.05065377,  0.0068942 ,  0.03104902, ..., -0.0335449 ,
         0.04959124,  0.00572576]])

In [68]:
from tensorflow import keras
model_1_keras = keras.models.load_model('model_1.keras')
model_1_keras.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               102600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 103,004
Trainable params: 103,004
Non-trainable params: 0
_________________________________________________________________


In [72]:
# model_1_keras.predict(features)
res = np.argmax(model_1_keras.predict(features), axis = -1)
res

array([0], dtype=int64)